## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-07-17-55 +0000,bbc,US drops charges against doctor accused of des...,https://www.bbc.com/news/articles/cy0w1p0wq87o
1,2025-07-13-07-16-33 +0000,nyt,"For Families of Air India Crash Victims, Repor...",https://www.nytimes.com/2025/07/13/world/asia/...
2,2025-07-13-04-28-08 +0000,nyt,Modi Wants More Indians to Speak Hindi. Some S...,https://www.nytimes.com/2025/07/13/world/asia/...
3,2025-07-13-03-34-49 +0000,bbc,EU and Mexico criticise Trump's proposed 30% t...,https://www.bbc.com/news/articles/cyvj13d9ylpo
4,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,india,5
9,air,5
26,trump,5
11,crash,4
80,was,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-07-13-07-16-33 +0000,nyt,"For Families of Air India Crash Victims, Repor...",https://www.nytimes.com/2025/07/13/world/asia/...,25
8,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...,25
23,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo,25
17,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,24
5,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,23


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,25,2025-07-13-07-16-33 +0000,nyt,"For Families of Air India Crash Victims, Repor...",https://www.nytimes.com/2025/07/13/world/asia/...
17,24,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
5,23,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
13,19,2025-07-12-21-11-05 +0000,bbc,The mushroom killer was obsessed with true cri...,https://www.bbc.com/news/articles/c0m8glx2zleo
4,13,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
21,13,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
26,11,2025-07-12-09-30-00 +0000,wsj,Pittsburgh is striving to turn its old steel m...,https://www.wsj.com/tech/ai/can-pittsburghs-ol...
6,10,2025-07-13-00-00-42 +0000,bbc,Putin's friend Gergiev set to conduct as Italy...,https://www.bbc.com/news/articles/czxw4n7vy70o
12,10,2025-07-12-21-49-47 +0000,bbc,Gaza hospital says 24 people killed near aid s...,https://www.bbc.com/news/articles/cp90v2ng70yo
20,10,2025-07-12-14-34-23 +0000,bbc,Israeli settlers kill Palestinians in West Ban...,https://www.bbc.com/news/articles/cm2mknzn99eo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
